# Experiment 02

- [ ] mask entries in larger dataset in long-format
- [ ] mask peptides based on their frequency in samples (probability of being observed)
- [ ] create training data set without masked values for each model
    - Denoising AE
    - FNN based on embeddings (Collaborative Filtering)
    - VAE
- [ ] restrict to only a training data split of consective data: Increase number of samples.
    - focus on best reconstruction performance
    - mean comparison

### Collaborative Filtering model
- Cannot accomodate iid assumption of statistical test in current setup for embedding vectors.
    - if pretrained model should be applied to an new batch of replicates (with a certain condition)
      one would need to find a way to initialize the sample embeddings without fine-tuning the model

In [ ]:
from pprint import pprint
from src.nb_imports import *

from pathlib import Path
from src import metadata


import logging
from src.logging import setup_logger

logger = setup_logger(logger=logging.getLogger('vaep'))
logger.info("Experiment 02")

figures = {}  # collection of ax or figures

ADD_TENSORBOARD = False

In [ ]:
# None takes all
N_SAMPLES = 500

## Raw data

In [ ]:
# only some sample have many missings

FN_PEPTIDE_INTENSITIES = config.FOLDER_DATA / \
    'df_intensities_N_07813_M01000'  # all
FN_PEPTIDE_INTENSITIES = config.FOLDER_DATA / \
    'df_intensities_N_07637_M01000'  # 60%
FN_PEPTIDE_INTENSITIES = config.FOLDER_DATA / \
    'df_intensities_N_07285_M01000'  # 90%

In [ ]:
analysis = AnalyzePeptides(fname=FN_PEPTIDE_INTENSITIES, nrows=None)
analysis.df.columns.name = 'peptide'
analysis.log_transform(np.log2)
analysis

In [ ]:
# some date are not possible in the indices
rename_indices_w_wrong_dates = {'20161131_LUMOS1_nLC13_AH_MNT_HeLa_long_03': '20161130_LUMOS1_nLC13_AH_MNT_HeLa_long_03',
                                '20180230_QE10_nLC0_MR_QC_MNT_Hela_12': '20180330_QE10_nLC0_MR_QC_MNT_Hela_12',
                                '20161131_LUMOS1_nLC13_AH_MNT_HeLa_long_01': '20161130_LUMOS1_nLC13_AH_MNT_HeLa_long_01',
                                '20180230_QE10_nLC0_MR_QC_MNT_Hela_11': '20180330_QE10_nLC0_MR_QC_MNT_Hela_11',
                                '20161131_LUMOS1_nLC13_AH_MNT_HeLa_long_02': '20161130_LUMOS1_nLC13_AH_MNT_HeLa_long_02'}
analysis.df.rename(index=rename_indices_w_wrong_dates, inplace=True)

### Select N consecutive samples

In [ ]:
# sort index
analysis.df.sort_index(inplace=True)
analysis.df_all = analysis.df

In [ ]:
import random
N_SAMPLES = min(len(analysis.df), N_SAMPLES) if N_SAMPLES else len(analysis.df)

random.seed(42)


def get_consecutive_data_indices(df, n_samples=N_SAMPLES):
    index = df.sort_index().index
    start_sample = len(index) - n_samples
    start_sample = random.randint(0, start_sample)
    return df.loc[index[start_sample:start_sample+n_samples]]


_attr_name = f'df_{N_SAMPLES}'
setattr(analysis, _attr_name, get_consecutive_data_indices(analysis.df_all))
print("Training data stored under:", _attr_name)
analysis.df = getattr(analysis, _attr_name)
analysis.df

In [ ]:
assert not analysis.df._is_view

- biological stock differences in PCA plot. Show differences in models. Only see biological variance

In [ ]:
d_meta = metadata.get_metadata_from_filenames(analysis.df.index)
analysis.df_meta = pd.DataFrame.from_dict(
    d_meta, orient='index')
analysis.df_meta

Use to find date parsing errors, used for renaming above.

In [ ]:
# invalid_dates = pd.to_datetime(analysis.df_meta.date, errors='coerce').isna()
# display(analysis.df_meta.loc[invalid_dates])
# {i : i for i in analysis.df_meta.loc[invalid_dates].index} # to rename

In [ ]:
analysis.df_meta.describe()

See rare instrument types (potential labeling errors)

In [ ]:
N_MIN_INSTRUMENT = 10
ms_instruments = analysis.df_meta.ms_instrument.value_counts()
ms_instruments = ms_instruments[ms_instruments > N_MIN_INSTRUMENT].index
mask = ~analysis.df_meta.ms_instrument.isin(ms_instruments)
analysis.df_meta.loc[mask]

### PCA plot of raw data

In [ ]:
import itertools
from sklearn.impute import SimpleImputer
X = SimpleImputer().fit_transform(analysis.df)
X = vaep.pandas._add_indices(X, analysis.df)
assert X.isna().sum().sum() == 0

pca = analyzers.run_pca(X)
cols = list(pca.columns)

fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(
    15, 20), constrained_layout=True)
fig.suptitle('First two Principal Components of entire dataset', fontsize=30)


# by instrument
ax = axes[0]
pca['ms_instrument'] = analysis.df_meta['ms_instrument'].astype('category')
for name, group in pca.groupby('ms_instrument'):
    ax.scatter(x=group[cols[0]], y=group[cols[1]], label=name)
ax.set_title('by category', fontsize=18)
ax.legend(loc='center right', bbox_to_anchor=(1.11, 0.5))

# by dates
ax = axes[1]
ax.set_title('by date', fontsize=18)
path_collection = analyzers.scatter_plot_w_dates(
    ax, pca, dates=analysis.df_meta.date, errors='raise')
path_collection = analyzers.add_date_colorbar(path_collection, fig)

## Long format

- Data in long format: (peptide, sample_id, intensity)
- no missing values kept
- 

In [ ]:
def get_long_format(self, colname_values='intensity', inplace=False):
    df = self.df
    df_long = df.unstack().dropna().to_frame(colname_values)
    df_long = df_long.reset_index('Sample ID')
    if inplace:
        self.df_long = df_long
        return
    return df_long


get_long_format(analysis, inplace=True)
analysis.df_long.head()

In [ ]:
assert analysis.df_long.isna().sum().sum() == 0, "There are still missing values in the long format."

In [ ]:
def get_wide_format(self, columns='Sample ID', name_values='intensity', inplace=False):
    df_wide = self.df_long.pivot(columns=columns, values=name_values)
    df_wide = df_wide.T
    if inplace:
        self.df_wide = df_wide
        return
    return df_wide


get_wide_format(analysis, inplace=True)
analysis.df_wide.head()

In [ ]:
assert analysis.df_wide.isna().sum().sum() > 0, "There are no missing values left in the wide format"

### Sampling peptides by their frequency (important for later)

- higher count, higher probability to be sampled into training data
- missing peptides are sampled both into training as well as into validation dataset
- everything not in training data is validation data

In [ ]:
# freq_per_peptide = analysis.df.unstack().to_frame('intensity').reset_index(1, drop=True)
freq_per_peptide = analysis.df_long['intensity']
freq_per_peptide = freq_per_peptide.notna().groupby(level=0).sum()

In [ ]:
# df_long = analysis.df.unstack().to_frame('intensity').reset_index(1)
analysis.df_train = analysis.df_long.groupby(
    by='Sample ID').sample(frac=0.95, weights=freq_per_peptide, random_state=42)
analysis.df_train = analysis.df_train.reset_index().set_index([
    'Sample ID', 'peptide'])
analysis.df_train

## MultiIndex 

- use mulitindex for obtaining validation split

In [ ]:
analysis.df_long = analysis.df_long.reset_index(
).set_index(['Sample ID', 'peptide'])
analysis.df_long.head()

In [ ]:
analysis.indices_valid = analysis.df_long.index.difference(
    analysis.df_train.index)
analysis.df_valid = analysis.df_long.loc[analysis.indices_valid]

In [ ]:
assert len(analysis.df_long) == len(analysis.df_train) + len(analysis.df_valid)

## Setup DL

In [ ]:
import vaep.model as vaep_model
from vaep.cmd import get_args

BATCH_SIZE, EPOCHS = 128, 30
args = get_args(batch_size=BATCH_SIZE, epochs=EPOCHS,
                no_cuda=False)  # data transfer to GPU seems slow
kwargs = {'num_workers': 2, 'pin_memory': True} if args.cuda else {}

# torch.manual_seed(args.seed)
device = torch.device("cuda" if args.cuda else "cpu")
device

print(args, device, sep='\n')

Fastai default device for computation

In [ ]:
import fastai.torch_core
# # device = torch.device('cpu')
# # fastai.torch_core.defaults.device = torch.device('cpu')
# device = fastai.torch_core.defaults.device
# device
torch.cuda.is_available()

In [ ]:
fastai.torch_core.defaults

## Collaboritive filtering model

In [ ]:
from fastai.collab import CollabDataLoaders, MSELossFlat, Learner

analysis.collab = Analysis()
collab = analysis.collab
collab.columns = 'peptide,Sample ID,intensity'.split(',')

Create data view for collaborative filtering

- currently a bit hacky as the splitter does not support predefinded indices (create custum subclass providing splits to internal methods?)

- Use the [`CollabDataLoaders`](https://docs.fast.ai/collab.html#CollabDataLoaders)  similar to the [`TabularDataLoaders`](https://docs.fast.ai/tabular.data.html#TabularDataLoaders).
- Use the [`IndexSplitter`](https://docs.fast.ai/data.transforms.html#IndexSplitter) and provide splits to whatever is used in `CollabDataLoaders`


In [ ]:
collab.df_train = analysis.df_train.reset_index()
collab.df_valid = analysis.df_valid.reset_index()
collab.df_train.head()

In [ ]:
collab.df_valid.head()

In [ ]:
assert (collab.df_train.intensity.isna().sum(),
        collab.df_valid.intensity.isna().sum()) == (0, 0), "Remove missing values."

Hacky part uses training data `Datasets` from dataloaders to recreate a custom `DataLoaders` instance

In [ ]:
collab.dl_train = CollabDataLoaders.from_df(
    collab.df_train, valid_pct=0.0, user_name='Sample ID', item_name='peptide', rating_name='intensity', bs=args.batch_size, device=device)
collab.dl_valid = CollabDataLoaders.from_df(
    collab.df_valid, valid_pct=0.0, user_name='Sample ID', item_name='peptide', rating_name='intensity', bs=args.batch_size, 
    shuffle=False, device=device)
collab.dl_train.show_batch()

In [ ]:
from fastai.data.core import DataLoaders
collab.dls = DataLoaders(collab.dl_train.train, collab.dl_valid.train)
if args.cuda:
    collab.dls.cuda()

In [ ]:
collab.dl_valid.show_batch()

In [ ]:
len(collab.dls.classes['Sample ID']), len(collab.dls.classes['peptide'])

In [ ]:
len(collab.dls.train), len(collab.dls.valid) # mini-batches

Alternatively to the hacky version, one could use a factory method, but there the sampling/Splitting methods would need to be implemented (not using [`RandomSplitter`](https://docs.fast.ai/data.transforms.html#RandomSplitter) somehow)

 - [`TabDataLoader`](https://docs.fast.ai/tabular.core.html#TabDataLoader)
 - uses [`TabularPandas`](https://docs.fast.ai/tabular.core.html#TabularPandas)
 
 > Current problem: No custom splitter can be provided

In [ ]:
# valid_idx = [analysis.df_long.index.get_loc(key=key) for key in analysis.indices_valid]
# splitter = IndexSplitter([valid_idx])
# splitter(collab.df.index)

# # replace in CollabDataloaders for getting splits
# # or directly in TabularCollab
# CollabDataLoaders??

In [ ]:
# # drop NAs before?
#
# from fastai.tabular.all import *
# from fastai.tabular.data import TabularDataLoaders
# collab.dls = TabularDataLoaders.from_df(
#     df=analysis.df_long.reset_index(), 
#     procs=[Categorify],
#     valid_idx=valid_idx,
#     cat_names=['Sample ID', 'peptide'],
#     y_names=['intensity'],
#     with_cont=False,
#     y_block=TransformBlock(),
#     bs=64)
# collab.dls.show_batch()
# # Problem: this return a second empty df - > would need to adapt model.

A brief check that the values match roughly

In [ ]:
# from numpy.testing import assert_almost_equal
# UPTODECIMAL = 5
# assert_almost_equal(
#     collab.dls.valid_ds['intensity'].values, 
#     analysis.df_long.iloc[valid_idx]['intensity'],
#     decimal=UPTODECIMAL
# )
# print(f"Values match up to the {UPTODECIMAL} decimal.")

### Model

In [ ]:
collab.model_args = {}
collab.model_args['n_samples'] = len(collab.dls.classes['Sample ID'])
collab.model_args['n_peptides'] = len(collab.dls.classes['peptide'])
collab.model_args['dim_latent_factors'] = 20
collab.model_args['y_range'] = (
    int(analysis.df_train['intensity'].min()), int(analysis.df_train['intensity'].max())+1)

print("Args:")
pprint(collab.model_args)


model = vaep_model.DotProductBias(**collab.model_args)
learn = Learner(dls=collab.dls, model=model, loss_func=MSELossFlat())
if args.cuda:
    learn.cuda()
learn.summary()

### Training

In [ ]:
learn.fit_one_cycle(5, 5e-3)

### Evaluation

In [ ]:
collab.dls.valid_ds.items

In [ ]:
# import pandas as pd
# dtype = pd.CategoricalDtype(collab.dls.classes['peptide'], ordered=False)
# pd.Categorical.from_codes(codes=collab.dls.valid_ds.items['Sample ID'], dtype=dtype)

In [ ]:
# show False does not return results..
res = learn.show_results(show=True)  # something similar with return

In [ ]:
df_pred = collab.df_valid.copy()
df_pred['intensity_pred_collab'] = pd.Series(pred.flatten().numpy(), index=collab.dls.valid_ds.items.index)

def cast_object_to_category(df):
    _dtypes = df.dtypes
    _columns = _dtypes[_dtypes == 'object'].index
    return df.astype({col: 'category' for col in _columns})

df_pred = cast_object_to_category(df_pred)
df_pred

In [ ]:
# # Adapt to get prediction Dataframe
# encodings, pred, target = learn.get_preds(
#     with_input=True)  # per default validation data
# pred_df = pd.DataFrame([{'Sample ID': collab.dls.classes['Sample ID'][obs[0]], 'peptide': collab.dls.classes['peptide']
#                          [obs[1]], 'intensity_pred_collab': pred_intensity.item(), 'intensity': orig_intensity.item() } for obs, pred_intensity, orig_intensity in zip(encodings, pred, target)])
# # pred_df = pred_df.pivot(index='Sample ID', columns='peptide')
# pred_df.sort_values(by=['Sample ID', 'peptide'])

In [ ]:
(abs(target - pred)).sum() / len(target)

## Denoising Autoencoder (DAE)

In [ ]:
from fastcore.transform import Transform
class NormalizeShiftedMean(Transform):
    def setups(self, df, shift_mu=0.5, scale_var=2.0):
        #settattr()
        self.mean, self.std = df.mean(), df.std()
        self.shift_mu, self.scale_var = shift_mu, scale_var
    def encodes(self, x): 
        x = (x - self.mean) / self.std 
        return x / self.scale_var + self.shift_mu
    
    def decodes(self, x): 
        x = (x - self.shift_mu) * self.scale_var
        return x * self.std + self.mean

norm_shifted = NormalizeShiftedMean()
norm_shifted.setup(analysis.df)

norm_shifted(analysis.df.iloc[:5])

In [ ]:
# revert format
analysis.df_train = analysis.df_train['intensity'].unstack()
analysis.df_valid = analysis.df_valid['intensity'].unstack()

In [ ]:
from fastai.data.transforms import Normalize
norm = Normalize.from_stats(analysis.df.mean(),analysis.df.std())

In [ ]:
from fastai.tabular.all import *

In [ ]:
from fastai.tabular.core import TabularPandas
# FillMissing adds mask -> need to adapt network?
# https://docs.fast.ai/tabular.core.html#FillStrategy
procs = [norm, FillMissing]
cont_names = list(set(analysis.df))
cat_names=None
y_names=None
splits=None
y_block=None
to = TabularPandas(analysis.df_train, procs, cat_names, cont_names, y_names, splits=splits, y_block=y_block)
to.items

In [ ]:
assert len(to.valid) == 0

In [ ]:

# class MaskData(Callback):
def after_batch(batch):
    return (batch, batch)
#         x_cat, x_cont, y = self.batch
#         # first needs to be masked somehow
#         self.learner.batch = x_cont, x_cont

In [ ]:
args.batch_size
dls = to.dataloaders(shuffle_train=True, shuffle=False, bs=args.batch_size, after_batch=after_batch)
xb = dls.train.one_batch()
len(xb)

In [ ]:
# xb[0] # now there are imputed values and a mask

In [ ]:
# from vaep.transform import ShiftedStandardScaler

# args_ae = {}
# args_ae['SCALER'] = StandardScaler
# args_ae['SCALER'] = ShiftedStandardScaler

# # select initial data: transformed vs not log transformed
# scaler = args_ae['SCALER'](scale_var=2).fit(analysis.df_train)
# # five examples from validation dataset
# scaler.transform(analysis.df_train).describe(percentiles=[0.025, 0.975])

In [ ]:
# from torchvision import transforms
# from torch.utils.data import DataLoader
# from vaep.io.datasets import PeptideDatasetInMemoryMasked

# # ToDo: replace with helper class (see below)
# tf_norm = None  # replace with Normalizer

# dataset_train = PeptideDatasetInMemoryMasked(
#     data=scaler.transform(analysis.df_train.values))
# dataset_valid = PeptideDatasetInMemoryMasked(
#     data=scaler.transform(analysis.df_valid.values))
# dl_train = DataLoader(dataset_train, batch_size=args.batch_size, shuffle=True)
# dl_valid = DataLoader(dataset_valid, batch_size=args.batch_size, shuffle=False)

In [ ]:
M = analysis.df_train.shape[-1]
model = vaep_model.Autoencoder(n_features=M, n_neurons=int(
    M/2), last_decoder_activation=None, dim_latent=30).double()
criterion = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-2

optimizer = torch.optim.Adam(
    model.parameters(),
    lr=learning_rate,
)

In [ ]:
xb = learn.dls.train.one_batch()[:learn.dls.train.n_inp]
type(xb)

In [ ]:
# valid_idx = [analysis.df_long.index.get_loc(key=key) for key in analysis.indices_valid]
# splitter = IndexSplitter([valid_idx])
learn = Learner(dls=dls, model=model, loss_func=vaep_model.loss_function)
learn.summary()

In [ ]:
learn.fit_one_cycle(5, 5e-3)

## Variational Autoencoder (VAE)

Save final prediction values with comparison.

In [ ]:
pred_df.to_csv(config.FOLDER_DATA / f"{config.FOLDER_DATA}_valid_pred")